In [ ]:
!pip install transformers torch accelerate -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer

In [ ]:
import pandas as pd

df = pd.read_csv("aug_data.txt", sep="|")

In [ ]:
for i in range(len(df['intent'])):
  df['intent'][i]=df['intent'][i].strip()

In [ ]:
# Load the data from the JSON files
import json
with open('surprise.data', 'r') as data_file:
    data = [json.loads(line) for line in data_file]

with open('surprise.solution', 'r') as solution_file:
    solutions = [json.loads(line) for line in solution_file]

In [ ]:
# Create a dictionary to map indoml_id to intents
from sklearn.model_selection import train_test_split

intent_map = {item['indoml_id']: item['intent'] for item in solutions}

# Split data into train and test sets (2:1) stratified by intent
indoml_ids = [item['indoml_id'] for item in data]
intents = [intent_map[indoml_id] for indoml_id in indoml_ids]

utt = [item['utt'] for item in data]

num_classes = len(set(intents))
print("Number of classes")
print(num_classes)

train_data, test_data, train_labels, test_labels = train_test_split(
    utt, intents, test_size=0.1, random_state=42, stratify=intents
)

Number of classes
150


In [ ]:
# convert that to dataframe
import pandas as pd
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)
# add labels as a column
train_data['labels'] = train_labels
test_data['labels'] = test_labels



,0,labels
129,Will there be any sunny days this week?,weather
184,Is there any chance of thunderstorms today?,weather
285,What is the probability of rain tomorrow?,weather
567,What is the weather like in Tokyo right now?,weather
735,Will it be sunny or cloudy tomorrow morning?,weather
791,Is it going to snow this winter?,weather
812,What is the weather forecast for this weekend?,weather
1144,Will there be any thunderstorms tonight?,weather
1364,What is the current temperature outside?,weather
1555,Will it snow tonight?,weather


In [ ]:
list(train_data[train_data['labels'] == 'reset settings'][0][:])

['Kindly reinstate the settings to their original condition.',
 'Can you reset everything to how it was when I first started using it?',
 "I'd like to wipe the slate clean and begin anew.",
 'change back to default settings.',
 "I'd like to reset everything to how it was when I first got you.",
 "I'd be grateful if you could reset my settings to their factory defaults right away.",
 'Could you please reset my settings to their default values so I can start anew?',
 "Let's restore the default configurations for a fresh start.",
 'I yearn for the simplicity of my initial settings',
 'start fresh with new settings',
 'Please undo all recent changes and restore the original settings.',
 'Would you kindly reset my settings to how they were when I first got this device?',
 "Let's restore the device to its original state."]

In [ ]:
sum(['traffic' in i.lower() for i in list(train_data[train_data['labels'] != 'traffic'][0][:])])

2

In [ ]:
[i for i in list(train_data[train_data['labels'] != 'traffic'][0][:]) if 'traffic' in i.lower() ]

['Will there be a lot of foot traffic outside the store during the holiday season?',
 'She might be late, maybe she got stuck in traffic.']

In [ ]:
list(train_data[train_data['labels'] == 'whisper mode'][0][:])

['I wish we could communicate quietly without being overheard.',
 'please keep your voice down.',
 'switch to low volume now.',
 'Can you refrain from shouting?',
 'I wish we were in a place where we could whisper our secrets without worrying about being overheard.',
 'engage mute mode immediately.',
 'whisper sweet nothings into my ear.',
 "I'm trying to concentrate; could you please whisper?",
 'If you could kindly reduce the volume to a soft whisper, it would be greatly appreciated.',
 'I think we should hush our voices.',
 'i need you to whisper this message.',
 "Could you please speak more softly? It's bothering me.",
 'I suggest we converse softly.',
 'quiet down and speak softly.']

In [ ]:
['traffic' in i.lower() for i in list(train_data[train_data['labels'] == 'traffic'][0][:])]

[True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True]

In [ ]:
list(train_data[train_data['labels'] == 'w2'][0][:])

['Can someone please guide me on how to retrieve my W2 form?',
 'Could you kindly inform me where to obtain my W2 form?',
 "Where can I find the form that shows my employer's name and my annual wages for tax purposes?",
 'Do you know if my employer will send me my W-2 forms via email this year?',
 "I'm trying to locate my W-2 form so I can submit it to the IRS before the deadline.",
 "I've been trying to track down my W2 form for weeks; could you please help me locate it?",
 "Is there any way to download my W2 form directly from my employer's website?",
 'How can I access my W-2 form if I misplaced it?',
 'I need help finding my W2 form so I can complete my tax return.',
 'How do I know which box to fill out on the W2 form for my state taxes?',
 'I am diligently searching for my W2 form to submit my taxes; could you please share the location of mine?',
 'Can you please tell me where I can find my lost W-2 form?',
 'Where can I download a blank W-2 form for free?',
 'I would like to kno

In [ ]:
# Define the BERT model and tokenizer
# keep `ignore_mismatched_sizes=True` so that the classification layer is randomly initialized
model_name = "cartesinus/bert-base-uncased-amazon-massive-intent"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained("trained_model")
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)

In [ ]:
# Tokenize the input data
train_encodings = tokenizer(
    train_data,
    truncation=True,
    padding=True,
    max_length=32,
    return_tensors='pt'    #return type is pytorch tensor
)

test_encodings = tokenizer(
    test_data,
    truncation=True,
    padding=True,
    max_length=32,
    return_tensors='pt'
)

In [ ]:
# Convert labels to numeric values
label_map = {intent: i for i, intent in enumerate(set(train_labels))}
train_labels = [label_map[label] for label in train_labels]
test_labels = [label_map[label] for label in test_labels]

In [ ]:
# Create PyTorch datasets
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels = None, test = False):
        self.encodings = encodings
        self.labels = labels
        self.test = test

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.test == False: item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = IntentDataset(train_encodings, train_labels)
test_dataset = IntentDataset(test_encodings, test_labels)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./intent_classification',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps", # use 'epoch' for evaluating every epoch
    logging_steps=100,
    eval_steps=100,
    save_total_limit=5,
    learning_rate=1e-4,
    num_train_epochs=20,
    logging_dir='./logs',
)

In [ ]:
# Create a trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Step,Training Loss,Validation Loss
100,3.102900,1.805471
200,0.275800,1.943988
300,0.182600,1.915223
400,0.106100,1.973182
500,0.102600,2.294327
600,0.103000,2.183093
700,0.075300,2.187203
800,0.058000,2.340619
900,0.047200,2.382209
1000,0.042100,2.369909


TrainOutput(global_step=3380, training_loss=0.1315976498659546, metrics={'train_runtime': 743.58, 'train_samples_per_second': 145.243, 'train_steps_per_second': 4.546, 'total_flos': 1778359629312000.0, 'train_loss': 0.1315976498659546, 'epoch': 20.0})

In [ ]:
# Predict on the test set
raw_predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(raw_predictions.predictions, axis=1)

# Convert labels back to original intents
predicted_intents = [list(label_map.keys())[list(label_map.values()).index(label)] for label in predicted_labels]

# Print one sample prediction
sample_idx = 1
sample_text = test_data[sample_idx]
sample_intent = predicted_intents[sample_idx]

print(f"Sample Text: {sample_text}")
print(f"Predicted Intent: {sample_intent}")

Sample Text:   Nice to meet you!
Predicted Intent: thank you


In [ ]:
with open('massive_test.data', 'r') as solution_file:
    final = [json.loads(line) for line in solution_file]


# Split data into train and test sets (2:1) stratified by intent
final_indoml_ids = [item['indoml_id'] for item in final]
final_utt = [item['utt'] for item in final]

final_encodings = tokenizer(
    final_utt,
    truncation=True,
    padding=True,
    max_length=32,
    return_tensors='pt'    #return type is pytorch tensor
)

final_dataset = IntentDataset(final_encodings, test = True)

final_raw_predictions = trainer.predict(final_dataset)

final_predicted_labels = np.argmax(final_raw_predictions.predictions, axis=1)

# Convert labels back to original intents
final_predicted_intents = [list(label_map.keys())[list(label_map.values()).index(label)] for label in final_predicted_labels]

# Print one sample prediction

In [ ]:
'''with open('surprise.data', 'r') as solution_file:
    final = [json.loads(line) for line in solution_file]


# Split data into train and test sets (2:1) stratified by intent
final_indoml_ids = [item['indoml_id'] for item in final]
final_utt = [item['utt'] for item in final]

final_encodings = tokenizer(
    final_utt,
    truncation=True,
    padding=True,
    max_length=32,
    return_tensors='pt'    #return type is pytorch tensor
)

final_dataset = IntentDataset(final_encodings, test = True)

final_raw_predictions = trainer.predict(final_dataset)

final_predicted_labels = np.argmax(final_raw_predictions.predictions, axis=1)

# Convert labels back to original intents
final_predicted_intents = [list(label_map.keys())[list(label_map.values()).index(label)] for label in final_predicted_labels]

# Print one sample prediction'''

In [ ]:
final_predicted_intents

In [ ]:
'''with open('surprise.solution', 'r') as solution:
    soln = [json.loads(line) for line in solution]'''

In [ ]:
'''solnx=[]
for i in soln:
  solnx.append(i['intent'])'''

In [ ]:
'''final_predicted_intents'''

In [ ]:
'''len(set(final_predicted_intents)),len(set(solnx))'''

(150, 150)

In [ ]:
'''for i in range(len(final_predicted_intents)):
  final_predicted_intents[i]=final_predicted_intents[i].strip()'''

In [ ]:
'''label_map = {intent: i for i, intent in enumerate(set(solnx))}'''

In [ ]:
'''final_predicted_intents = [label_map[label] for label in final_predicted_intents]'''

In [ ]:
'''solnx = [label_map[label] for label in solnx]'''

In [ ]:
'''final_predicted_intents'''

In [ ]:
'''solnx'''

In [ ]:
'''from sklearn.metrics import accuracy_score, precision_score, f1_score'''

In [ ]:
'''accuracy = accuracy_score(solnx, final_predicted_intents)
print('accuracy ',accuracy)

precision = precision_score(solnx, final_predicted_intents,average='weighted')
print('precision',precision)

f1 = f1_score(solnx, final_predicted_intents,average='weighted')
print('f1_score ',f1)
'''

accuracy  0.8371886120996441
precision 0.8405116878019865
f1_score  0.834337924851306


In [ ]:
trainer.save_model("trained_model")

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from concurrent.futures import ThreadPoolExecutor

# Function to get BERT embeddings for a given text
def get_bert_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Function to calculate cosine similarity
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Function to predict labels in parallel
def predict_labels_parallel(test_embedding, train_label_embeddings):
    similarities = [calculate_similarity(test_embedding, label_embedding) for label_embedding in train_label_embeddings]
    predicted_label_index = similarities.index(max(similarities))
    return train_labels[predicted_label_index]

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Training data
train_input_texts = ["Train input text 1", "Train input text 2", ...]
train_labels = ["Train Label 1", "Train Label 2", ...]

# Test data
test_input_texts = ["Test input text 1", "Test input text 2", ...]
test_labels = ["Test Label 1", "Test Label 2", ...]

# Get embeddings for training data in parallel
with ThreadPoolExecutor() as executor:
    train_input_embeddings = list(executor.map(lambda text: get_bert_embedding(text, model, tokenizer), train_input_texts))
    train_label_embeddings = list(executor.map(lambda label: get_bert_embedding(label, model, tokenizer), train_labels))

# Get embeddings for test data in parallel
with ThreadPoolExecutor() as executor:
    test_input_embeddings = list(executor.map(lambda text: get_bert_embedding(text, model, tokenizer), test_input_texts))

# Predict labels for test data in parallel
with ThreadPoolExecutor() as executor:
    predicted_labels = list(executor.map(lambda test_embedding: predict_labels_parallel(test_embedding, train_label_embeddings), test_input_embeddings))

# Calculate accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print("Accuracy on Test Data:", accuracy)
